In [139]:
import numpy as np
import warnings

# warnings.simplefilter("ignore", category=RuntimeWarning)

PRINT = False

In [140]:
def check_closure(x1, x2, eps):
    return np.linalg.norm(x1 - x2) <= eps

# Метод Якоби
$a_{ii}x_i^{(k+1)}=f_i-\sum\limits_{j=1,i\neq j}^n a_{ij}x_j^{(k)}$

In [141]:
def get_yakobi_iter(prev_res, anti_diag_matrix, inv_diagonal_matrix,  f):
    if PRINT:
        print("Data:\n", anti_diag_matrix, "\n", prev_res)
        print("Mult res:\n", anti_diag_matrix @ prev_res)
        print("Sub:\n", f - anti_diag_matrix @ prev_res)
    return inv_diagonal_matrix @ (f - anti_diag_matrix @ prev_res)

# Метод последовательной верхней релаксации

$\left( D + \omega A_{L} \right)\dfrac{x^{(k + 1)} - x^{(k)}}{\omega} + Ax^{(k)} = f$

In [142]:
def get_upper_relax_iter(prev_res, omega, matrix, f):
    if PRINT:
        print("Data:\n", prev_res)
    x1 = np.zeros(prev_res.shape)
    for i in range(prev_res.shape[0]):
        s = 0
        for j in range(i):
            s += matrix[i, j] * x1[j]
        x1[i] -= omega * s

        s = 0
        for j in range(i + 1, prev_res.shape[0]):
            s += matrix[i, j] * prev_res[j]
        x1[i] -= omega * s

        x1[i] += (1 - omega) * prev_res[i] * matrix[i, i]

        x1[i] /= matrix[i, i]
    return x1

# Проверка

In [143]:
def main_cycle(method, eps, main_shape, *args):
    iter = 1
    prev_iter = np.ones(main_shape)
    cur_iter = prev_iter
    if PRINT:
        print(args)
    while True:
        prev_iter = cur_iter
        cur_iter = method(prev_iter, *args)
        if PRINT:
            print("cur_iter:\n",cur_iter)
        if check_closure(cur_iter, prev_iter, eps):
            break;
        iter += 1
    
    return cur_iter, iter

## Входные данные

In [144]:
A = np.matrix(
    "6.22 1.42 -1.72 1.91;"
    "1.42 5.33 1.11 -1.82;"
    "-1.72 1.11 5.24 1.42;"
    "1.91 -1.82 1.42 6.55"
)

f = np.matrix("7.53;"
              "6.06;"
              "8.05;"
              "8.06")


D = np.diag(np.diag(A))

anti_D = A - D
inv_D = np.linalg.inv(D)

eps = pow(10, -4)

## Якоби

In [145]:
cur_iter, iter = main_cycle(get_yakobi_iter, eps, f.shape, anti_D, inv_D, f)


cur_iter, np.linalg.solve(A, f), iter

(matrix([[1.28737763],
         [0.69362225],
         [1.62325748],
         [0.69598733]]),
 matrix([[1.28757167],
         [0.69342757],
         [1.62345151],
         [0.69579773]]),
 35)

## Метод последовательной верхней релаксации

In [146]:
import pandas as pd

df = pd.DataFrame(columns=["f", "omg", "iter"])

for omg in range(100, 199 + 1):
    omg /= 100
    cur_iter, iter = main_cycle(get_upper_relax_iter, eps, f.shape, omg, A, f)
    
    df.loc[len(df)] = [cur_iter, omg, iter]

df

,f,omg,iter
0,"[[-0.00010732358242283858], [9.694449489366695...",1.00,20
1,"[[-8.991362755044057e-05], [8.100965409856096e...",1.01,20
2,"[[-7.483337010268691e-05], [6.724362849332976e...",1.02,20
3,"[[-9.775541004308709e-05], [8.759916126725104e...",1.03,19
4,"[[-8.112494180208714e-05], [7.24891048955723e-...",1.04,19
...,...,...,...
95,"[[2.1817599281194186e-07], [-2.491960766292057...",1.95,241
96,"[[-3.2741566568484e-05], [-2.7295977901724836e...",1.96,302
97,"[[3.18853417280147e-06], [2.73541810570092e-05...",1.97,406
98,"[[-5.808294051427803e-06], [-3.041406791471035...",1.98,612


Как видим, метод последовательной верхней релаксации может быть как гораздо быстрее метода Якоби, так и сильно медленнее, если не повезёт взять неправильную $\omega$